# Tutorial: Training IC


The first thing is import the openmichrom module, to install openmm and openmichrom follow the instalation guide on (www..rice.edu)

In [1]:
import openmichrom as OM

The OM is a alias for OpenMichrom module, you can use any name you desire

in OpenMichroM initiation there are some variable to setup:

**timestep=0.01** (the time step using for integration, default is 0.01)<br>
**thermostat=0.1** (the Langevin thermostat, defaut value is 10, e.g., 0.1)<br>
**temperature=120** (Set the temperature of your simulation)<br>
**name='open-MiChroM'** (the simulation name)<br>

In [2]:
sim = OM.OpenMiChroM(name='single_chr', temperature=120, timestep=0.01)

Now you need to setup the plataform that you use, the options are:

**platform="cuda"** (remenber that you need install CUDA in your system)<br>
**GPU="0" (optional)** (if you have more than one GPU device, you can set the what gpu you wnat ["0", "1",...,"n"])<br>
**platform="cpu"**<br>
**platform="opencl"**<br>





In [3]:
sim.setup(platform="cuda")

Set the Folder name where output are save

In [4]:
sim.saveFolder('/home/antonio/single_chr/')

The next step is setup your chromosome sequence and initial configuration

The chromosome sequence is loaded using a file containing the type annotation of every bead, in 2-coloun form format:<br>
1 A1<br>
2 A1<br>
3 A2<br>
...<br>
700 B4<br>

In this process you define a spiral open structure with position saved in mypol variable as numpy array $[3:x]$, where $x$ is the lenght of your chromossome. 

In sim.type_list is set your type annotations, where:
0 : A1<br>
1 : A2<br>
2 : B1<br>
3 : B2<br>
4 : B3<br>
5 : B4<br>
6 : NA<br>




In [5]:
mychro = sim.create_springSpiral(type_list='/home/antonio/openmichrom1.0/Tutorial/single_chromosome/chr10_beads.txt')

In [6]:
print(mychro)
print(sim.type_list[:10]) #print the first 10 values

[[367.3932139  198.35136593 197.79432009]
 [367.40384406 198.72274184 198.72274184]
 [367.3932139  199.09411774 199.65116358]
 ...
 [278.84874274 172.10182396 201.50620703]
 [278.85865018 172.29430711 200.57913526]
 [278.88750799 172.48035101 199.65116358]]
[6, 6, 2, 2, 2, 2, 2, 2, 2, 2]


Load the initial structure into "sim" object

The center=True is optitonal and move your system to origin

In [7]:
sim.load(mychro, center=True)

You can save a file to check initial structure

the file is saved in .ndb format

The file is save in savefolder

In [8]:
sim.save(mode = 'ndb')

Now its time to include the forces of your system

Lets separete forçes in two sets:

homopolymer Potentials  


In [9]:
sim.addFENEBonds(k=30.0)
sim.addAngles(k=1.0)
sim.addRepulsiveSoftCore(Ecut=4.0)
sim.addSphericalConfinement(density=0.1, k=5)

18.638046406856677

Chromosome Potentials

In [10]:
sim.addTypetoType(mi=3.22, rc = 1.78)
sim.addLoops(mi=3.22, rc = 1.78, X=-1.612990,
                 filename=['/home/antonio/openmichrom1.0/Tutorial/single_chromosome/chr10_looplist.txt'])
sim.addIdealChromosome(mi=3.22, rc = 1.78, dcutl=3, dcutupper=500)

The simulation setup is complete!

Now I will create a file .cndb (binary) to save the frames

The file will be saved in output folder defined earlier

In [11]:
sim.initStorage('chr_10', mode='w')

The Open-MiChroM simulation works in blocks of simulations

for exemple, if want to run $1\times10^8$ steps, saving a frame each $10^3$ steps 

block = $10^3$ <br>
n_block = $10^5$ <br>


In [18]:
block = 10**3 
n_blocks = 10**5 

In [13]:
for _ in range(n_blocks):
    sim.doBlock(block, output=True) #makes output=False if won't print details about simularion 
    sim.save()  #save frame on cndb file

Number of exceptions:
First neighbor= 2711
Second neighbor= 2710
adding force  FENEBond 0
adding force  AngleForce 1
Add exclusions for RepulsiveSoftCore force (1 neighbor)
adding force  RepulsiveSoftCore 2
adding force  SphericalConfinement 3
Add exclusions for TypetoType force (1 neighbor)
adding force  TypetoType 4
adding force  Loops 5
Add exclusions for IdealChromosome force (1 neighbor)
adding force  IdealChromosome 6
Positions... 
 loaded!
potential energy is 577.192801
bl=1 (i) pos[1]=[158.2 3.4 4.4] dr=18.70 t=10.0ps kin=51.96 pot=484.58 Rg=114.897 SPS=865 
bl=2 (i) pos[1]=[139.0 -0.4 4.2] dr=18.45 t=20.0ps kin=51.57 pot=392.78 Rg=97.100 SPS=1363 
bl=3 (i) pos[1]=[121.9 0.4 5.0] dr=18.54 t=30.0ps kin=51.70 pot=300.45 Rg=79.488 SPS=1349 
bl=4 (i) pos[1]=[104.9 1.0 6.0] dr=18.40 t=40.0ps kin=48.90 pot=211.41 Rg=62.475 SPS=1352 
bl=5 (i) pos[1]=[82.0 1.6 3.5] dr=16.55 t=50.0ps kin=39.12 pot=139.93 Rg=47.036 SPS=1329 
bl=6 (i) pos[1]=[66.0 -0.7 6.5] dr=14.84 t=60.0ps kin=31.34 pot

In [14]:
sim.storage[0].close() #close the cndb file

If you want to save a structure to visualize in vmd, chimera, etc.. You can save a ndb file.<br>

**The ndb interpreter for visualize in vmd/chimera is under construction<br>**

In [16]:
sim.save(mode="ndb")

To visualize the ndb files we can use the ndb converter that can be download 
in https://ndb.rice.edu/ndb-format